In [ ]:
import os

from PIL import Image
from scipy.ndimage import center_of_mass

import pandas as pd
import numpy as np
import imageio
import matplotlib.pyplot as plt

import PIL

In [ ]:
hands_pd = pd.read_csv("~/data/11khands/HandInfo.csv")
hands_pd

In [ ]:
def read_img(name, numpy=True, pil=False):
    img = imageio.imread(f"~/data/11khands/Hands/{name}")
    if numpy: return np.array(img)
    return img

def imshow(img, ax):
    if not isinstance(img, np.ndarray):
        img = img / 2 + 0.5     # unnormalize
        img = img.numpy()
        img = np.transpose(img, (1, 2, 0))
    ax.imshow(img)

def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 100, 100)
    return x

In [ ]:
img = read_img("Hand_0000422.jpg")
          
com_x = 0
com_y = 0
for i in range(3):
    com = center_of_mass(255 - img[:,:,i])
    com_x += 1/3 * com[1]
    com_y += 1/3 * com[0]
    
print(com_x, com_y)

plt.imshow(img)
plt.plot(com_x, com_y, 'or')
plt.show()

In [ ]:
def transform_raw_hand_image(img):
    com_x, com_y = 0, 0
    for i in range(3):
        com = center_of_mass(255 - img[:,:,i])
        com_x += 1/3 * com[1]
        com_y += 1/3 * com[0]
    com_x, com_y = int(com_x), int(com_y)
    
    box_x = max(0, min(400, com_x-600))
    box_y = max(0, com_y-600)
    box = (box_x, box_y, box_x + 1200, box_y + 1200)

    pil = PIL.Image.fromarray(img)
    crop = pil.crop(box)
    crop_res = crop.resize((100, 100), PIL.Image.BILINEAR)
    
    return crop_res

transform_raw_hand_image(read_img("Hand_0000854.jpg"))

In [ ]:
for f in os.listdir("~/data/11khands/Hands"):
    img = transform_raw_hand_image(read_img(f))
    img.save(f"~/data/11khands/small/unk/{f}")

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(14, 6))
imshow(read_img("Hand_0011740.jpg"), ax[0])
imshow(read_img("Hand_0000003.jpg"), ax[1])
imshow(read_img("Hand_0000004.jpg"), ax[2])
imshow(read_img("Hand_0000005.jpg"), ax[3])
plt.show()